In [9]:
import bs4
import urllib
import pandas as pd
import tqdm

In [2]:
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [3]:
links=['https://tamilvideosda.com/tamil-2021-videos-songs.html','https://tamilvideosda.com/tamil-2020-videos-songs.html','https://tamilvideosda.com/tamil-2019-videos-songs.html','https://tamilvideosda.com/tamil-2018-videos-songs.html','https://tamilvideosda.com/tamil-2017-videos-songs.html','https://tamilvideosda.com/tamil-2016-videos-songs.html','https://tamilvideosda.com/tamil-2015-videos-songs.html','https://tamilvideosda.com/tamil-hd-video-songs.html','https://tamilvideosda.com/atoz.php','https://tamilvideosda.com/tamil-album-songs.html']

In [4]:
new_data_1=[]
for li in links:
    i=1
    print(li)
    while True:
        mainsoup = bs4.BeautifulSoup(urllib.request.urlopen(f'{li}?get-page={i}'), 'html.parser')
        i+=1
        tmp=mainsoup.find_all("div", {"class": "f"})
        if(len(tmp)>0):
            for x in tmp:
                for song in x.findAll('a'):
                    new_data_1.append([song.string,song.get('href')])
        else:
            break

https://tamilvideosda.com/tamil-2021-videos-songs.html
https://tamilvideosda.com/tamil-2020-videos-songs.html
https://tamilvideosda.com/tamil-2019-videos-songs.html
https://tamilvideosda.com/tamil-2018-videos-songs.html
https://tamilvideosda.com/tamil-2017-videos-songs.html
https://tamilvideosda.com/tamil-2016-videos-songs.html
https://tamilvideosda.com/tamil-2015-videos-songs.html
https://tamilvideosda.com/tamil-hd-video-songs.html
https://tamilvideosda.com/atoz.php
https://tamilvideosda.com/tamil-album-songs.html


In [7]:

df = pd.DataFrame(new_data_1, columns=['level1','level1_link'])
df['Occur'] = df.groupby('level1')['level1'].transform('size')
df_filtered = df[(df['Occur'] ==1) | ( (df['Occur'] >1) & (~df['level1_link'].str.endswith('xhtml')) ) ]
print(len(df),len(df_filtered))

2173 1236


In [16]:
l2_df=pd.DataFrame(columns=['level1','level1_link','level2','level2_link'])

In [18]:
for i,r in tqdm.tqdm(df_filtered.iterrows(), total=df_filtered.shape[0]):
    li=r['level1_link']
    mainsoup = bs4.BeautifulSoup(urllib.request.urlopen(f'https://tamilvideosda.com{li}'), 'html.parser')
    tmp=mainsoup.find_all("div", {"class": "f"})
    if(len(tmp)>0):
        for x in tmp:
            for song in x.findAll('a'):
                l2_df.loc[len(l2_df.index)] = [r['level1'],r['level1_link'],song.string,song.get('href')] 


100%|██████████| 1236/1236 [04:32<00:00,  4.54it/s]


In [20]:
l3_df=pd.DataFrame(columns=['level1','level1_link','level2','level2_link','level3','level3_link'])

In [21]:
for i,r in tqdm.tqdm(l2_df.iterrows(), total=l2_df.shape[0]):
    li=r['level2_link']
    try:
        mainsoup = bs4.BeautifulSoup(urllib.request.urlopen(f'https://tamilvideosda.com{li}'), 'html.parser')
    except:
        mainsoup = bs4.BeautifulSoup(urllib.request.urlopen(li), 'html.parser')
        
    tmp=mainsoup.find_all("div", {"class": "f"})
    if(len(tmp)>0):
        for x in tmp:
            for song in x.findAll('a'):
                l3_df.loc[len(l3_df.index)] = [r['level1'],r['level1_link'],r['level2'],r['level2_link'],song.string,song.get('href')] 
    

100%|██████████| 5211/5211 [21:59<00:00,  3.95it/s]


In [29]:
final_data=l3_df[(l3_df['level3_link'].str.startswith('https://tamilvideosda.com'))&(~l3_df['level3'].str.endswith('3gp'))]

In [30]:
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect("data.sqlite")

In [31]:
final_data[['level1','level2','level3','level3_link']].to_sql('data', con, if_exists='replace', index=True)